In [ ]:
__nbid__ = '0040'
__author__ = 'Brian Merino <brian.merino@noirlab.edu>, Vinicius Placco <vinicius.placco@noirlab.edu>'
__version__ = '20241209' # yyyymmdd; Versión DataSpamp de este notebook
__keywords__ = ['gmos','gemini','stars','dragons']

# Reducción de fotometría de campo estrellado Gemini GMOS usando Dragons Python API
#### Adaptado de https://dragons.readthedocs.io/projects/gmosimg-drtutorial/en/v3.2.1/ex1_gmosim_starfield_api.html

<a class="anchor" id="attribution"></a>
# Avisos Legales y atribuciones

Avisos Legales
-----------
Tome en cuenta que usar el Astro Data Lab constituye un acuerdo con nuestros [Avisos Legales](https://datalab.noirlab.edu/disclaimers.php) mínimos.

Reconocimientos
---------------
Si ud. usa el **Astro Data Lab** en sus publicaciones de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios de datos proveeidos por el Astro Data Lab, el cual es parte del Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) del NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc."(Asociación de Universidaddes para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

Si utiliza **SPARCL junto con la plataforma de Astro Data Lab** (por medio de JupyterLab, línea de comando o interfaz de la web) en su publicación de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios o datos proporcionados por el "SPectra Analysis and Retrievable Catalog Lab" (SPARCL) (Laboratorio de Análisis y Catálogo Recuperable de Espectros) y el Astro Data Lab, ambos pertenecientes al Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) de NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc." (Asociación de Universidades para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

En cualquiera de los casos, **por favor cite las siguientes publicaciones**:

* Publicación del concepto de Data Lab: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Descripción general del Astro Data Lab: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411.

Si hace referencia al Jupyterlab / Jupyter notebooks de Data Lab, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097.

Si publica en una revista de la AAS, agregue también la palabra clave `\facility{Astro Data Lab}`

Y si está usando SPARCL, por vor agregue también `\software{SPARCL}` y cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576.

La biblioteca de NOIRLab mantiene [listas de reconocimientos apropiados](https://noirlab.edu/science/about/scientific-acknowledgments) para usar cuando se hacen publicaciones utilizando los recursos, servicios o datos del Laboratorio.

---- **Versión en Inglés** ----


# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.


<a class = "Anchor" id = "importaciones"> </a>
# Importar bibliotecas de Python

In [ ]:
import warnings
import glob
import os
import shutil

from gempy.adlibrary import dataselect
from gempy.utils import logutils

from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce

from astropy.io import fits
from astropy.wcs import WCS
from astropy.utils.exceptions import AstropyWarning

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

warnings.simplefilter('ignore', category=AstropyWarning)

<a class = "Anchor" id = "Preparar"> </a>
# Prepare el directorio de trabajo

Si tiene algún archivo intermedio que se creó al ejecutar este código en el pasado, deberá eliminarlos de su directorio de trabajo. La siguiente celda define una función de limpieza que eliminará todos los archivos de ajuste de su directorio de trabajo. Esta función se volverá a llamar al final del tutorial, dejándote solo con el producto final. Por defecto, esta función eliminará todos los archivos en el directorio de trabajo. Si hay archivos que se han reducido previamente que le gustaría conservar, establezca `save_reduced = 1` al llamar a la función.

In [ ]:
def clean_up(save_reduced=0):
    # ¿El directorio de calibraciones ya existe?
    caldb_Exist = os.path.exists('./calibrations') 
    
    if caldb_Exist:
        shutil.rmtree('./calibrations', ignore_errors=True)

    # Eliminar archivos de registro y lista existentes.
    work_dir_path = os.getcwd()
    work_dir = os.listdir(work_dir_path)

    for item in work_dir:
        if item.endswith(".log") or item.endswith(".list"):
            os.remove(os.path.join(work_dir_path, item))
    
    # A continuación, eliminaremos todos los archivos de ajuste existentes, excepto los archivos reducidos previamente, dependiendo de lo que establezca save_reduced.
    if save_reduced:
        all_files_0 = glob.glob('*.fits')
        save = dataselect.select_data(all_files_0, [], ['PROCESSED'])
        
        for s in save:
            os.remove(os.path.join(work_dir_path,s))

    else:
        all_files_0 = glob.glob('*.fits')
        for a in all_files_0:
            os.remove(os.path.join(work_dir_path,a))

In [ ]:
clean_up(save_reduced=0)

<a class = "Anchor" id = "Acerca"> </a>
# Sobre el conjunto de datos

Los datos utilizados para este tutorial son una secuencia aturdida en un campo estrellado.

La siguiente tabla contiene un resumen del conjunto de datos:


| Tipo de observación | Nombre del archivo (s) | Propósito y exposición (segundos) |
| : --- | : --- | : ---: |
| Ciencia | N20170614S0201-205 | 10 s, banda I |
| Sesgo | N20170613S0180-184 |  |
| Sesgo | N20170615S0534-538 |  |
| Twilight Flats | N20170702S0178-182 | 40 a 16 s, I-Band |

<a class = "Anchor" id = "descargar_data"> </a>
# Descargar los datos

Descarga de imágenes de banda I del Archivo de Géminis al directorio de trabajo actual. Este paso solo necesita ser ejecutado una vez.

Si ejecuta este notebook por primera vez y necesita descargar el conjunto de datos, configure la variable "download = true". El notebook no se volverá a descargar el conjunto de datos si está configurado en falso. Esto se volverá particularmente útil si ejecuta los notebooks más de una vez.

In [ ]:
%%bash 

# Crear archivo que enumera se ajuste a los archivos a descargar
echo "\
http://archive.gemini.edu/file/N20170613S0180.fits
http://archive.gemini.edu/file/N20170613S0181.fits
http://archive.gemini.edu/file/N20170613S0182.fits
http://archive.gemini.edu/file/N20170613S0183.fits
http://archive.gemini.edu/file/N20170613S0184.fits
http://archive.gemini.edu/file/N20170614S0201.fits
http://archive.gemini.edu/file/N20170614S0202.fits
http://archive.gemini.edu/file/N20170614S0203.fits
http://archive.gemini.edu/file/N20170614S0204.fits
http://archive.gemini.edu/file/N20170614S0205.fits
http://archive.gemini.edu/file/N20170615S0534.fits
http://archive.gemini.edu/file/N20170615S0535.fits
http://archive.gemini.edu/file/N20170615S0536.fits
http://archive.gemini.edu/file/N20170615S0537.fits
http://archive.gemini.edu/file/N20170615S0538.fits
http://archive.gemini.edu/file/N20170702S0178.fits
http://archive.gemini.edu/file/N20170702S0179.fits
http://archive.gemini.edu/file/N20170702S0180.fits
http://archive.gemini.edu/file/N20170702S0181.fits
http://archive.gemini.edu/file/N20170702S0182.fits\
" > gmos_im_star.list

In [ ]:
%%bash

download="True"

if [ $download == "True" ]; then
    wget --no-check-certificate -N -q -i gmos_im_star.list

else
    echo "Skipping download. To download the data set used in this notebook, set download=True."
fi

<a class = "Anchor" id = "dragons_logger"> </a>
# Configuración del Logger Dragons

Dragons viene con un administrador de calibración local que utiliza las mismas reglas de asociación de calibración que el archivo del Observatorio Géminis. Esto permite reducir las solicitudes a una base de datos local de peso ligero para las calibraciones procesadas que coinciden cuando sea necesario para reducir un conjunto de datos.

Esto le dice al sistema dónde colocar la base de datos de calibración. Esta base de datos realizará un seguimiento de las calibraciones procesadas que le enviaremos.

In [ ]:
logutils.config(file_name='gmos_data_reduction.log')
caldb = cal_service.set_local_database()
caldb.init("w")

In [ ]:
all_files = glob.glob('N2017*[0-9].fits')
all_files.sort()

<a class = "FASCOR" ID = "FILE_LISTS"> </a>
# Crear listas de archivos

Este conjunto de datos contiene marcos de ciencia y calibración. Para algunos programas, podría tener diferentes objetivos observados y tiempos de exposición dependiendo de cómo organice sus datos sin procesar.

La tubería de reducción de datos de Dragons no organiza los datos por usted. Tienes que hacerlo. Dragons proporciona herramientas para ayudarlo con eso.

El primer paso es crear listas que se utilizarán en el proceso de reducción de datos. Para eso, usamos DataSelect. Consulte el [DataSElect](https://dragons.readthedocs.io/projects/recipe-system-users-manual/en/stable/supptools/dataselect.html?highlight=dataSelect) documentación para obtener detalles sobre su uso.

** Lista de prejuicios **

In [ ]:
list_of_biases = dataselect.select_data(
    all_files,
    ['BIAS'],
    []
)

** Lista de pisos **

Si su conjunto de datos tiene pisos obtenidos con más de un filtro, puede agregar la expresión --expr 'filtre_name == "i" para obtener solo los pisos obtenidos dentro de la banda I. Por ejemplo:

In [ ]:
list_of_flats = dataselect.select_data(
     all_files,
     ['FLAT'],
     [],
     dataselect.expr_parser('filter_name=="i"')
)

** Lista de datos científicos **

In [ ]:
list_of_science = dataselect.select_data(
    all_files,
    [],
    ['CAL'],
    dataselect.expr_parser('(observation_class=="science" and filter_name=="i")')
)

<a class = "Anchor" id = "Master_bias"> </a>
# Crea un sesgo maestro
Comenzamos la reducción de datos creando un sesgo maestro para los datos científicos. Se puede crear y agregar a la base de datos de calibración utilizando los comandos a continuación. El sesgo maestro tendrá el nombre del primer sesgo con el sufijo _bias.fits

In [ ]:
reduce_bias = Reduce()
reduce_bias.files.extend(list_of_biases)
reduce_bias.runr()

<a class = "Anchor" id = "Master_flat"> </a>
# Crea un campo plano maestro

Las imágenes planas crepusculares se utilizan para producir un plano de maestros de imágenes y el resultado se agrega a la base de datos de calibración.

El Master Flat tendrá el nombre del primer archivo plano de Twilight con el sufijo _flat.fits.

In [ ]:
reduce_flats = Reduce()
reduce_flats.files.extend(list_of_flats)
reduce_flats.runr()

<a class = "Anchor" id = "Reduce_science"> </a>
# Reducir las imágenes de la ciencia

Una vez que nuestros archivos de calibración se procesan y se agregan a la base de datos, podemos ejecutar reducir en nuestros datos científicos.

Este comando generará sesgo y archivos planificados y los apilará. Si se necesita un marco marginal, este comando aplicará la corrección. La imagen apilada tendrá el sufijo _stack.

Las unidades de pila de salida están en electrones (palabra clave de encabezado Bunit = electrones). La pila de salida se almacena en un archivo de ajuste de extensión múltiple (MEF). La señal científica está en la extensión "Sci", la varianza está en la extensión "var" y el plano de calidad de datos (máscara) está en la extensión "DQ".

Cada imagen científica reducida tendrá el nombre original con el sufijo _image.fits.

In [ ]:
reduce_science = Reduce()
reduce_science.files.extend(list_of_science)
reduce_science.runr()

<A class = "Anchor" id = "Imagen de visualización"> </y>
# Muestra la imagen apilada

In [ ]:
image_file = "N20170614S0201_image.fits"
hdu_list = fits.open(image_file)
wcs = WCS(hdu_list[1].header)
hdu_list.info()

In [ ]:
image_data = fits.getdata(image_file, ext=1)
print(image_data.shape)

In [ ]:
plt.figure(figsize = (10,10))
plt.subplot(projection=wcs)
plt.imshow(image_data,cmap='gray',norm=LogNorm(vmin=0.01, vmax=1000000),origin='lower')
plt.xlabel('Ascensión derecha [HH: MM: SS]',fontsize=14,fontweight='atrevido')
plt.ylabel('Declinación [grado]',fontsize=14,fontweight='atrevido')
plt.show()

<a class = "Anchor" id = "Clean-Up"> </a>
# Opcional: elimine las calibraciones duplicadas y elimine los datos sin procesar (líneas de incommento antes de ejecutar)

In [ ]:
# clean_up (save_reduced = 1)